In [8]:
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
import numpy as np
from pathlib import Path

The goal of this step is mainly to convert the source Leica file into individual 3D tiff files. For some data, the current python reader cannot load the channel dimenstion and the Z dimension correctly. So, we have to re-order the data. We only need such dimension fix on the very first dataset. For future data, just try to run without dimension fix. If the output is not correct, then re-run with dimension fix.

In [ ]:
# flag to select dimension_fix or not
use_dim_fix = False

# specific chanel index
# currently, we assume the second channel (index=1) is CD31 and 
# the third channel (index=2) is Col IV (note: index is zero-based)
get_channels = [1, 2]

# add you file path
src = Path("/path/to/string_vessel_data/")

# get all LIF images
filenames = sorted(src.glob("*.lif"))
print(len(filenames))

# specific where you want to save the splitted files
out_path_base = Path("/path/to/output_folder/")
out_path_3d = out_path_base / Path("split_3d")
out_path_pred = out_path_base / Path("pred")  # used for saving model prediction

out_path_base.mkdir(parents=True, exist_ok=True)
out_path_3d.mkdir(parents=True, exist_ok=True)
out_path_pred.mkdir(parents=True, exist_ok=True)

for fn in filenames:
    reader = AICSImage(fn)
    scene_list = reader.scenes
    print("scene_list: ", scene_list)

    # loop through all scenes
    for sname in scene_list:
        reader.set_scene(sname)
        # get image data
        img = reader.get_image_dask_data("CZYX", T=0).compute()

        # check if dimension_fix is necessary
        if use_dim_fix:
            img_re = np.zeros_like(img)
            counter_z = 0
            counter_c = 0
            for c in range(img.shape[0]):
                for z in range(img.shape[1]):
                    img_re[counter_c, counter_z, :, :] = img[c, z, :, :]
                    counter_c += 1
                    if counter_c == img.shape[0]:
                        counter_c = 0
                        counter_z += 1
            im = img_re[get_channels, :, :, :]
        else:
            im = img[get_channels, :, :, :]

        # clean up filenames to get rid of spaces
        sname = sname.replace("/", "_")

        # save individual multi-channel Tiff files
        out_fn = out_path_3d / f"{fn.stem}_{sname}.tiff"
        OmeTiffWriter.save(im, out_fn, dim_order="CZYX")
